# 고차원 데이터 시각화: 유방암 데이터셋 탐색

이 노트북은 특성의 개수가 많은 **고차원 데이터(High-Dimensional Data)**를 효과적으로 시각화하고 탐색하는 방법을 다룹니다. 특히 `scikit-learn`의 유방암 데이터셋(30개 특성)을 사용하여, 각 특성의 분포와 특성 간의 상관관계를 시각적으로 분석하는 과정을 보여줍니다.

**학습 목표:**
1.  `seaborn`과 `matplotlib`을 사용하여 고차원 데이터의 특성 분포를 클래스별로 시각화합니다.
2.  특성 간의 상관관계를 히트맵으로 시각화하여 데이터의 구조를 파악합니다.
3.  `pairplot`과 같은 시각화 도구가 고차원 데이터에 적용될 때의 한계를 이해합니다.

### 1. 라이브러리 임포트 및 데이터 로드

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris, load_breast_cancer 
import seaborn as sns 
import pandas as pd 
import numpy as np

# 한글 폰트 설정 (Windows 기준)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 유방암 데이터셋 로드
cancer = load_breast_cancer()

# 데이터를 Pandas DataFrame으로 변환하여 다루기 쉽게 합니다.
df_cancer = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df_cancer['target'] = cancer.target

print("데이터프레임 상위 5행:")
print(df_cancer.head())

### 2. 특성 관계 시각화 (Pairplot) - 고차원 데이터의 한계

`seaborn.pairplot`은 데이터프레임 내의 모든 숫자형 특성 쌍에 대한 산점도와 각 특성의 분포(히스토그램 또는 KDE)를 한 번에 보여주는 강력한 도구입니다. `hue` 옵션을 사용하면 타겟 클래스별로 색상을 구분하여 시각화할 수 있습니다.

**하지만, 특성의 개수가 많아질수록 `pairplot`은 비효율적이 됩니다.** 유방암 데이터셋은 30개의 특성을 가지고 있으므로, `pairplot`을 그리면 `30 * 30 = 900`개의 차트가 생성되어 시각적으로 복잡하고 실행 시간이 매우 오래 걸립니다. 따라서 이 데이터셋에는 적합하지 않습니다.

```python
# 유방암 데이터셋에 pairplot을 적용하면 매우 많은 차트가 생성되어 실행 시간이 오래 걸립니다.
# sns.pairplot(df_cancer, hue="target") 
# plt.show()
```

**참고**: 특성이 4개인 `iris` 데이터셋과 같이 특성 수가 적은 경우에는 `pairplot`이 매우 유용합니다.

### 3. 클래스별 특성 분포 시각화 (Histograms)

각 특성의 분포를 악성(target=0)과 양성(target=1) 클래스별로 비교하는 히스토그램을 그려보면, 어떤 특성이 두 클래스를 잘 구분하는지 파악하는 데 도움이 됩니다.

In [ ]:
# 데이터를 악성(0)과 양성(1)으로 분리
malignant_data = cancer.data[cancer.target == 0]
benign_data = cancer.data[cancer.target == 1]

print(f"악성 데이터 형태: {malignant_data.shape}")
print(f"양성 데이터 형태: {benign_data.shape}")

# 30개의 특성에 대한 히스토그램을 그리기 위해 15행 2열의 서브플롯을 생성
fig, axes = plt.subplots(15, 2, figsize=(10, 20))
ax = axes.ravel() # 2차원 axes 배열을 1차원으로 평탄화

for i in range(cancer.data.shape[1]): # 특성의 개수만큼 반복 (30개)
    # 각 특성(컬럼)의 값 범위를 기준으로 50개의 구간(bins)을 생성
    _, bins = np.histogram(cancer.data[:, i], bins=50)
    
    # 악성 데이터의 히스토그램 (보라색)
    ax[i].hist(malignant_data[:, i], bins=bins, color='purple', alpha=0.5, label='악성')
    # 양성 데이터의 히스토그램 (초록색)
    ax[i].hist(benign_data[:, i], bins=bins, color='green', alpha=0.5, label='양성')
    
    # 각 서브플롯의 제목을 특성 이름으로 설정
    ax[i].set_title(cancer.feature_names[i])
    ax[i].set_yticks(()) # y축 눈금 제거

# 첫 번째 서브플롯에만 범례와 축 레이블 추가
ax[0].set_xlabel('특성 값')
ax[0].set_ylabel('빈도')
ax[0].legend()

fig.tight_layout() # 서브플롯 간의 간격을 자동으로 조절
plt.show()

### 4. 특성 간 상관관계 시각화 (Heatmap)

히트맵은 특성들 간의 상관계수를 색상으로 표현하여, 어떤 특성들이 서로 강한 관계를 가지는지 한눈에 파악할 수 있게 해줍니다.

- **상관계수**: 두 특성 간의 선형 관계 강도를 나타내는 지표. -1(강한 음의 상관관계)부터 1(강한 양의 상관관계)까지의 값을 가집니다. 0에 가까우면 관계가 거의 없습니다.
- **`df.corr()`**: DataFrame의 모든 숫자형 컬럼 간의 상관계수 행렬을 계산합니다.

In [ ]:
# 상관관계 행렬 계산
correlation_matrix = df_cancer.corr()

print("상관관계 행렬 (일부):")
print(correlation_matrix.iloc[:5, :5]) # 상위 5x5만 출력하여 확인

# 히트맵 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix,
            annot=True,      # 셀에 상관계수 값 표시
            cmap='coolwarm', # 색상 맵 (양의 상관관계는 붉은색, 음의 상관관계는 푸른색)
            fmt='.2f',       # 표시될 숫자의 소수점 자리수 지정
            linewidths=.5    # 셀 사이에 선 추가
           )
plt.xticks(rotation=45, ha='right') # x축 레이블 회전
plt.yticks(rotation=0)
plt.title('특성 간 상관관계 히트맵')
plt.tight_layout() # 레이블 겹침 방지
plt.show()

### 결론

고차원 데이터는 그 자체로 이해하기 어렵지만, 히스토그램과 히트맵과 같은 시각화 도구를 통해 데이터의 분포와 특성 간의 관계를 파악할 수 있습니다. 이러한 탐색적 데이터 분석(EDA)은 데이터의 복잡성을 줄이고(차원 축소), 모델 학습에 더 적합한 특성을 선택하는 데 중요한 통찰력을 제공합니다.